# <center>Laboratorium Analiza i bazy danych </center>
# Marcin Piątek 400038

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [32]:
import main

import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
import pickle
from pandas.core.algorithms import isin

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

# Zadanie 1

In [33]:
zap =  f'''SELECT
                f.title,
                f.length
            FROM
                film f 
            WHERE
                f.length = {84}'''

pd.read_sql(zap, con=connection)

,title,length
0,Daisy Menagerie,84
1,Gone Trouble,84
2,Guys Falcon,84
3,High Encino,84
4,Honey Ties,84
5,Idols Snatchers,84
6,Island Exorcist,84
7,Jeepers Wedding,84
8,Packer Madigan,84
9,Pluto Oleander,84


# Zadanie 2

In [34]:
# pd.read_sql('select city.city  from city', con=connection)
miasto = 'Zaria'
zap = f'''SELECT
                c.city,
                cst.first_name,
                cst.last_name
            FROM
                customer cst
                    JOIN address a ON cst.address_id = a.address_id
                    JOIN city c ON a.city_id = c.city_id
            WHERE
                c.city = '{miasto}' '''

pd.read_sql(zap, con=connection)
 



,city,first_name,last_name
0,Zaria,Constance,Reid


# Zadanie 3

In [35]:
zap =   f'''SELECT
                AVG(amount)
            FROM
                payment'''

pd.read_sql(zap, con=connection)

,avg
0,4.200606


# Zadanie 4

In [36]:
zap =   f'''SELECT
                cat.name as category,
                COUNT(f.*)
            FROM film f
                JOIN film_category fc ON f.film_id = fc.film_id
                JOIN category cat ON cat.category_id = fc.category_id
            GROUP BY
                cat.name'''

pd.read_sql(zap, con=connection)      

,category,count
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


# Zadanie 5

In [37]:
zap =   f'''SELECT
                co.country,
                COUNT(cst.*)
            FROM customer cst
                JOIN address a ON cst.address_id = a.address_id
                JOIN city c ON c.city_id = a.city_id
                JOIN country co ON co.country_id = c.country_id
            GROUP BY
                co.country_id'''

pd.read_sql(zap, con=connection)   

,country,count
0,"Virgin Islands, U.S.",1
1,Hungary,1
2,Bahrain,1
3,Russian Federation,28
4,Greece,2
...,...,...
103,Pakistan,5
104,Holy See (Vatican City State),1
105,Philippines,20
106,Anguilla,1


# Zadanie 6

In [38]:
zap =   f'''SELECT 
                store.store_id,  
                a.address,
                staff.first_name as menager_first_name,
                staff.last_name as menager_last_name,
                COUNT(c.*)
            FROM 
                store
                    JOIN address a ON store.address_id = a.address_id
                    JOIN staff ON store.manager_staff_id = staff.staff_id
                    JOIN customer c ON store.store_id = c.store_id                    
            GROUP BY 
                store.store_id,  
                a.address,
                staff.first_name,
                staff.last_name 
            HAVING 
                COUNT(c.*) BETWEEN 100 AND 300'''
 
pd.read_sql(zap, con=connection)   

,store_id,address,menager_first_name,menager_last_name,count
0,2,28 MySQL Boulevard,Jon,Stephens,273


# Zadanie 7

In [39]:
zap =   f'''SELECT 
                cst.first_name,
                cst.last_name,
                SUM(f.length)
            FROM
                customer cst
                    INNER JOIN rental rent ON cst.customer_id = rent.customer_id 
                    INNER JOIN inventory inv ON inv.inventory_id = rent.inventory_id
                    INNER JOIN film f ON f.film_id = inv.film_id
                    
            GROUP BY
                cst.first_name,
                cst.last_name
            HAVING
                SUM(f.length) > 200'''

pd.read_sql(zap, con=connection)

,first_name,last_name,sum
0,Scott,Shelley,2916
1,Tim,Cary,4476
2,Samuel,Marlow,2291
3,Vera,Mccoy,2275
4,Nelson,Christenson,2119
...,...,...,...
594,Erica,Matthews,2727
595,Stanley,Scroggins,3694
596,Ellen,Simpson,2977
597,Patsy,Davidson,3513


# Zadanie 8

In [40]:
zap =   f'''SELECT
                f.title,
                AVG(p.amount)
            FROM
                film f
                    JOIN inventory inv ON inv.film_id = f.film_id  
                    JOIN rental rent ON rent.inventory_id = inv.inventory_id    
                    JOIN payment p ON rent.rental_id = p.rental_id              
            GROUP BY
                f.title'''

pd.read_sql(zap, con=connection)

,title,avg
0,Frontier Cabin,5.990000
1,Arachnophobia Rollercoaster,4.772609
2,Cruelty Unforgiven,1.156667
3,Intolerable Intentions,5.704286
4,Monsoon Cause,6.101111
...,...,...
953,Reef Salute,2.171818
954,Bugsy Song,5.101111
955,Orient Closer,4.754706
956,Brotherhood Blanket,4.294348


# Zadanie 9

In [41]:
zap =   f'''SELECT
                c.name as category,
                AVG(f.length)
            FROM film f
                JOIN film_category fc ON f.film_id = fc.film_id
                JOIN category c ON c.category_id = fc.category_id
            GROUP BY
                c.name'''
                
pd.read_sql(zap, con=connection)

,category,avg
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


# Zadanie 10

In [42]:
zap   = f'''SELECT
                *
            FROM    
                (SELECT
                    cat.name,
                    MAX(f.length) as max_length
                FROM
                    film f
                        JOIN film_category fc ON f.film_id = fc.film_id
                        JOIN category cat ON fc.category_id = cat.category_id
                GROUP BY
                    cat.name) as sel_1
            JOIN
                (SELECT
                    cat.name,
                    f.length as max_length,
                    f.title
                FROM film f
                        JOIN film_category fc ON f.film_id = fc.film_id
                        JOIN category cat ON fc.category_id = cat.category_id
                ORDER BY f.length DESC, f.title ASC) as sel_2                     
            USING(name, max_length)'''

pd.read_sql(zap, con=connection)

,name,max_length,title
0,Games,185,Chicago North
1,Comedy,185,Control Anthem
2,Action,185,Darn Forrester
3,Animation,185,Gangs Pride
4,Music,185,Home Pity
5,Travel,185,Muscle Bright
6,Animation,185,Pond Seattle
7,Sci-Fi,185,Soldiers Evolution
8,Travel,185,Sweet Brotherhood
9,Action,185,Worst Banger


# Zadanie 11

In [43]:
zap =   f'''SELECT
                title, 
                length 
            FROM 
                film
            ORDER BY 
                length DESC, title ASC
            LIMIT 11'''

pd.read_sql(zap, con=connection)


,title,length
0,Chicago North,185
1,Control Anthem,185
2,Darn Forrester,185
3,Gangs Pride,185
4,Home Pity,185
5,Muscle Bright,185
6,Pond Seattle,185
7,Soldiers Evolution,185
8,Sweet Brotherhood,185
9,Worst Banger,185


In [25]:
main.film_in_category(1)

,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
...,...,...,...
59,Uprising Uptown,English,Action
60,Waterfront Deliverance,English,Action
61,Werewolf Lola,English,Action
62,Women Dorado,English,Action


In [26]:
main.number_films_in_category(1)

,category,count
0,Action,64


In [27]:
main.number_film_by_length(90,100)

,length,count
0,90,5
1,91,6
2,92,11
3,93,8
4,94,4
5,95,2
6,96,2
7,97,4
8,98,8
9,99,8


In [28]:
main.client_from_city('Zaria')

,city,first_name,last_name
0,Zaria,Constance,Reid


In [48]:
main.avg_amount_by_length(50)

,length,avg
0,50,3.810598


In [30]:
main.client_by_sum_length(100)

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


In [49]:
main.category_statistic_length('Sci-Fi')

,category,avg,sum,min,max
0,Sci-Fi,108.196721,6600,51,185
